# Named Entity Recognition using CRF model
In Natural Language Processing (NLP) an Entity Recognition is one of the common problem. The entity is referred to as the part of the text that is interested in. In NLP, NER is a method of extracting the relevant information from a large corpus and classifying those entities into predefined categories such as location, organization, name and so on. 
Information about lables: 
* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon

        1. Total Words Count = 1354149 
        2. Target Data Column: Tag

#### Importing Libraries

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
#Reading the csv file
df = pd.read_csv('./data/ner_dataset.csv', encoding = "ISO-8859-1")

In [3]:
#Display first 10 rows
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 40302,the,NN,O
freq,1,52573,145807,887908


#### Observations : 
* There are total 47959 sentences in the dataset.
* Number unique words in the dataset are 35178.
* Total 17 lables (Tags).

In [5]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [6]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

There are lots of missing values in 'Sentence #' attribute. So we will use pandas fillna technique and use 'ffill' method which propagates last valid observation forward to next.

In [7]:
df = df.fillna(method = 'ffill')

In [8]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [9]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [10]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [11]:
len(sentences)

47959

In [12]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


Getting all the sentences in the dataset.

In [13]:
sentences = getter.sentences

In [14]:
len(sentences)

47959

#### Feature Preparation
These are the default features used by the NER in nltk. We can also modify it for our customization.

In [15]:
def word2features(sent, i):
    word = sent[i][0] # the current word
    postag = sent[i][1] # the POS tag of the current word

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(), # the current word (lowercase)
        'word[-3:]': word[-3:],  # 3-character suffix
        'word[-2:]': word[-2:],  # 3-character suffix
        'word.isupper()': word.isupper(), # is the word uppercase?
        'word.istitle()': word.istitle(), # is the word lowercase?
        'word.isdigit()': word.isdigit(), # it the word a number?
        'postag': postag, # the POS tag of the word
        'postag[:2]': postag[:2], # last 2 character of the POS tag of the word
    }
    if i > 0: # if not first word add features for previous word 
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else: # if first word of sentence there is no previous word
        features['BOS'] = True # beginning of sentence

    if i < len(sent)-1: # if not last word add features for next word
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # end of sentence

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [16]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [18]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False, verbose = True)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 38367/38367 [00:10<00:00, 3665.22it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 137917
Seconds required: 2.543

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=2.00  loss=1349118.68 active=136910 feature_norm=1.00
Iter 2   time=2.01  loss=1057618.64 active=135497 feature_norm=4.40
Iter 3   time=1.02  loss=826591.69 active=129828 feature_norm=3.85
Iter 4   time=5.21  loss=454513.09 active=131380 feature_norm=3.24
Iter 5   time=1.14  loss=380812.16 active=133164 feature_norm=4.08
Iter 6   time=1.13  loss=295551.90 active=131778 feature_norm=5.87
Iter 7   time=1.12  loss=256742.59 active=124478 feature_norm=7.20
Iter 8   time=1.13  loss=228644.67 active=118831 feature_norm=8.19
Iter 9   time=1.04  loss=197544.63 active=110749 feature_nor

/Users/wc72da/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100, verbose=True)

In [19]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

#### Evaluating the model performance.
There is much more O entities in data set, but we’re more interested in other entities. To account for this we’ll use averaged F1 score computed for all labels except for O. sklearn-crfsuite.metrics package provides some useful metrics for sequence classification task, including this one.

In [20]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-geo',
 'B-gpe',
 'B-org',
 'I-org',
 'B-tim',
 'B-per',
 'I-per',
 'I-tim',
 'I-geo',
 'B-nat',
 'I-nat',
 'B-art',
 'B-eve',
 'I-eve',
 'I-art',
 'I-gpe']

In [21]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted', labels =labels)
print(f1_score)

0.8511850975817504


In [22]:
report = flat_classification_report(y_test, y_pred)
print(report)

/Users/wc72da/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

       B-art       0.44      0.15      0.22        81
       B-eve       0.52      0.38      0.44        61
       B-geo       0.86      0.90      0.88      7521
       B-gpe       0.97      0.94      0.95      3219
       B-nat       0.70      0.21      0.32        34
       B-org       0.80      0.74      0.77      3972
       B-per       0.85      0.83      0.84      3374
       B-tim       0.92      0.88      0.90      4033
       I-art       0.25      0.07      0.11        55
       I-eve       0.38      0.19      0.25        53
       I-geo       0.82      0.80      0.81      1462
       I-gpe       0.93      0.62      0.74        42
       I-nat       1.00      0.08      0.15        12
       I-org       0.81      0.80      0.80      3362
       I-per       0.84      0.91      0.87      3483
       I-tim       0.81      0.79      0.80      1313
           O       0.99      0.99      0.99    177212

    accuracy              

## Let’s check what classifier learned

In [23]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-geo  -> I-geo   8.414277
B-nat  -> I-nat   7.899703
B-org  -> I-org   7.550725
I-org  -> I-org   7.439938
B-art  -> I-art   7.319828
B-per  -> I-per   7.310900
B-tim  -> I-tim   7.291974
B-eve  -> I-eve   7.140804
I-tim  -> I-tim   7.100317
B-gpe  -> I-gpe   7.048318
I-art  -> I-art   6.849460
I-geo  -> I-geo   6.663883
I-eve  -> I-eve   6.617047
I-gpe  -> I-gpe   6.331629
I-per  -> I-per   6.033221
O      -> B-per   4.762875
O      -> O       4.520906
I-nat  -> I-nat   4.269670
O      -> B-tim   3.225079
B-geo  -> B-tim   2.718923

Top unlikely transitions:
I-tim  -> B-org   -0.420681
B-art  -> O       -0.441651
B-art  -> B-per   -0.477492
I-art  -> B-per   -0.558382
B-tim  -> B-art   -0.574160
I-gpe  -> O       -0.612843
I-art  -> O       -0.623511
I-gpe  -> B-geo   -0.629051
B-eve  -> B-tim   -0.770985
I-art  -> B-tim   -0.787978
B-eve  -> O       -0.832436
B-art  -> B-geo   -1.045687
B-tim  -> B-gpe   -1.048469
I-geo  -> B-gpe   -1.078524
I-art  -> B-geo  

Theoretically transitions O -> I should be the most unlikely, but since they never occur, the `all_possible_transitions=False` flag makes sures they get ignored.

## Check the state features

In [24]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.158353 O        word.lower():last
8.095732 O        word.lower():month
6.864339 B-org    word.lower():philippine
6.591011 B-tim    word.lower():multi-candidate
6.545099 B-gpe    word.lower():niger
6.470383 B-gpe    word.lower():afghan
6.203209 B-per    word.lower():president
6.152053 B-geo    word.lower():caribbean
6.018412 B-org    -1:word.lower():rice
6.012577 B-tim    word.lower():2000
6.006212 B-geo    -1:word.lower():hamas
5.850307 B-tim    word.lower():one-year
5.814563 B-geo    word.lower():europe
5.795934 B-per    word.lower():prime
5.795603 B-gpe    word.lower():nepal
5.736389 B-org    word.lower():mid-march
5.683419 B-tim    word.lower():february
5.647005 B-tim    word.lower():january
5.597681 B-org    word.lower():hamas
5.499085 B-per    word.lower():obama
5.390410 B-per    word.lower():greenspan
5.380266 B-tim    word.lower():august
5.352231 B-per    word.lower():senator
5.328137 O        word.lower():internet
5.289210 O        word.lower():week
5.247250 B-t

# Exercise: Improve performance by adding/removing features

Come up with your own features to improve performance. You can add into the existing features, or remove features.

Some ideas:

- Use the shape of a word and other linguistic features (https://spacy.io/usage/linguistic-features)

(Tip: Store the shape of all words into a dictionary so that you do not have to invoke spaCy's method every time you encounter the same word)
- Look into nltk implementation of NER https://github.com/nltk/nltk/blob/42262c9a7cdcb6f44ac08aebd575b5d7bf85b6ea/nltk/chunk/named_entity.py